# LSSTCam pointing errors
# Looking at the results of homing the mount.

Craig Lage - 18-Apr-25

In [ ]:
import os
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst.summit.utils.tmaUtils import TMAEventMaker

In [ ]:
client = makeEfdClient()
eventMaker = TMAEventMaker()

## Get the MTMount state changes

In [ ]:
startDay = 20250527
endDay = 20250629
dayObs = startDay
enableDict = {}
homeDict = {}
while dayObs <= endDay:
    try:
        start = Time(f"{dayObsIntToString(dayObs)}T12:00:00")
        end = Time(f"{dayObsIntToString(calcNextDay(dayObs))}T12:00:00")
        states = getEfdData(
            client,
            "lsst.sal.MTMount.logevent_summaryState",
            columns=['summaryState'],
            begin=start,
            end=end
        )
        enables = states[states['summaryState'] == 2]
        
        homes = getEfdData(
        client,
        "lsst.sal.MTMount.command_homeBothAxes",
        begin=start,
        end=end
    )
        print(f"There were {len(enables)} enable events and {len(homes)} home events on {dayObs}")
        enableDict[dayObs] = enables
        homeDict[dayObs] = homes
        dayObs = calcNextDay(dayObs)
    except:
        dayObs = calcNextDay(dayObs)
        continue

In [ ]:
homeDict[20250530]

# First pass look at encoder offsets

In [ ]:
for plot in ['Offsets', 'Residuals']:
    fig, axs = plt.subplots(2, 3, figsize=(15, 10))
    plt.subplots_adjust(wspace = 0.5)
    for n, dayObs in enumerate([20250531, 20250601, 20250604]):
        events = eventMaker.getEvents(dayObs)
        myEvents = [e for e in events if ((e.type.name=='SLEWING') and (e.duration > 30.0))]
        print(f"for {dayObs}, there were {len(myEvents)} slews that were longer than 30 seconds")
    
        for event in myEvents:
            az = getEfdData(
                client,
                "lsst.sal.MTMount.azimuth",
                columns=['actualPosition', 'actualPositionTimestamp'],
                event=event, 
                prePadding = -2.0,
                postPadding = -2.0
            )
            el = getEfdData(
                client,
                "lsst.sal.MTMount.elevation",
                columns=['actualPosition', 'actualPositionTimestamp'],
                event=event, 
                prePadding = -2.0,
                postPadding = -2.0
            )
            encoder = getEfdData(
                client,
                "lsst.sal.MTMount.encoder",
                event=event, 
                prePadding = -2.0,
                postPadding = -2.0
            )
            azValues = np.asarray(az["actualPosition"])
            elValues = np.asarray(el["actualPosition"])
            dAz = abs(azValues[0] - azValues[-1])
            dEl = abs(elValues[0] - elValues[-1])
            print(event.seqNum, dAz, dEl)
            if (dAz > 10.0) and (dEl > 10.0):
                break
        elPos = getEfdData(
            client,
            "lsst.sal.MTMount.logevent_elevationInPosition",
            event=event, 
            prePadding = -2.0,
            postPadding = -2.0
        )
        azPos = getEfdData(
            client,
            "lsst.sal.MTMount.logevent_azimuthInPosition",
            event=event, 
            prePadding = -2.0,
            postPadding = -2.0
        )
        
        azimuthPosition = encoder['azimuthEncoderPosition0']
        azimuthPosition += encoder['azimuthEncoderPosition1']
        azimuthPosition += encoder['azimuthEncoderPosition2']
        azimuthPosition += encoder['azimuthEncoderPosition3']
        azimuthPosition /= 4.0
        encoder['azimuthPosition'] = azimuthPosition
        
        elevationPosition = encoder['elevationEncoderPosition0']
        elevationPosition += encoder['elevationEncoderPosition1']
        elevationPosition += encoder['elevationEncoderPosition2']
        elevationPosition += encoder['elevationEncoderPosition3']
        elevationPosition /= 4.0
        encoder['elevationPosition'] = elevationPosition
        if len(azPos) > 0:
            azInPos = Time(azPos.index[0], scale='utc') - TimeDelta(2.0, format='sec')
            az = az[az['actualPositionTimestamp'] < azInPos.unix_tai] 
        azValues = np.asarray(az["actualPosition"])
        azValTimes = np.asarray(az["actualPositionTimestamp"])
        azEncValues = np.asarray(encoder['azimuthPosition'])
        azEncTimes = np.asarray(encoder['azimuthEncoderPositionTimestamp0'])
        azEncInterp = np.interp(azValTimes, azEncTimes, azEncValues)
        az['encoderPosition'] = azEncInterp
    
        if len(elPos) > 0:
            elInPos = Time(elPos.index[0], scale='utc') - TimeDelta(2.0, format='sec')
            el = el[el['actualPositionTimestamp'] < elInPos.unix_tai] 
        elValues = np.asarray(el["actualPosition"])
        elValTimes = np.asarray(el["actualPositionTimestamp"])
        elEncValues = np.asarray(encoder['elevationPosition'])
        elEncTimes = np.asarray(encoder['elevationEncoderPositionTimestamp0'])
        elEncInterp = np.interp(elValTimes, elEncTimes, elEncValues)
        el['encoderPosition'] = elEncInterp
        if plot == "Offsets":
            axs[0][n].set_title(f"{dayObs}, Slew event {event.seqNum}")
            axs[0][n].scatter(azValues, azEncInterp)
            azFit = np.polyfit(azValues, azEncInterp, 1)
            #print(fit)
            axs[0][n].text(0.1, 0.8, f"Slope = {azFit[0]:.6f}", transform=axs[0][n].transAxes)
            axs[0][n].text(0.1, 0.7, f"Offset = {(azFit[1]*3600):.2f} arcseconds", transform=axs[0][n].transAxes)
            axs[0][n].set_xlabel("Azimuth (degrees)")
            axs[0][n].set_ylabel("Azimuth Encoder average(degrees)")
            axs[0][n].set_title(f"{dayObs}, Slew event {event.seqNum}")
            axs[1][n].scatter(elValues, elEncInterp)
            elFit = np.polyfit(elValues, elEncInterp, 1)
            #print(fit)
            axs[1][n].text(0.1, 0.8, f"Slope = {elFit[0]:.6f}", transform=axs[1][n].transAxes)
            axs[1][n].text(0.1, 0.7, f"Offset = {(elFit[1]*3600):.2f} arcseconds", transform=axs[1][n].transAxes)
            axs[1][n].set_xlabel("Elevation (degrees)")
            axs[1][n].set_ylabel("Elevation Encoder average(degrees)")
        elif plot == "Residuals":
            axs[0][n].set_title(f"{dayObs}, Slew event {event.seqNum}")
            azFit = np.polyfit(azValues, azEncInterp, 1)
            print(azFit)
            azFitVals = np.polyval(azFit, azValues)
            axs[0][n].scatter(azValues, (azEncInterp - azFitVals) * 3600.0)
            axs[0][n].text(0.1, 0.8, f"Slope = {azFit[0]:.6f}", transform=axs[0][n].transAxes)
            axs[0][n].text(0.1, 0.7, f"Offset = {(azFit[1]*3600):.2f} arcseconds", transform=axs[0][n].transAxes)
            axs[0][n].set_xlabel("Azimuth (degrees)")
            axs[0][n].set_ylabel("Azimuth Encoder residuals(arcseconds)")
            axs[0][n].set_title(f"{dayObs}, Slew event {event.seqNum}")
            axs[0][n].set_ylim(-1000, 1000)
            elFit = np.polyfit(elValues, elEncInterp, 1)
            print(elFit)
            elFitVals = np.polyval(elFit, elValues)
            axs[1][n].scatter(elValues, (elEncInterp - elFitVals) * 3600.0)
            axs[1][n].text(0.1, 0.8, f"Slope = {elFit[0]:.6f}", transform=axs[1][n].transAxes)
            axs[1][n].text(0.1, 0.7, f"Offset = {(elFit[1]*3600):.2f} arcseconds", transform=axs[1][n].transAxes)
            axs[1][n].set_xlabel("Elevation (degrees)")
            axs[1][n].set_ylabel("Elevation Encoder residuals(arcseconds)")
            axs[1][n].set_ylim(-1000, 1000)
    
    fig.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Encoder_{plot}_26Jun25.png")

In [ ]:
encoder['elevationEncoderPosition0'].plot()
encoder['elevationEncoderPosition1'].plot()
encoder['elevationEncoderPosition2'].plot()
encoder['elevationEncoderPosition3'].plot()
el['actualPosition'].plot()
plt.axvline(elInPos.isot)

# Checking if encoder timestamps always match - they don't.

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(15, 10))
plt.subplots_adjust(wspace = 0.5)
for n, dayObs in enumerate([20250531, 20250601, 20250604]):
    events = eventMaker.getEvents(dayObs)
    myEvents = [e for e in events if ((e.type.name=='SLEWING') and (e.duration > 30.0))]
    print(f"for {dayObs}, there were {len(myEvents)} slews that were longer than 30 seconds")

    for event in myEvents:
        az = getEfdData(
            client,
            "lsst.sal.MTMount.azimuth",
            columns=['actualPosition', 'actualPositionTimestamp'],
            event=event, 
            prePadding = -2.0,
            postPadding = -2.0
        )
        el = getEfdData(
            client,
            "lsst.sal.MTMount.elevation",
            columns=['actualPosition', 'actualPositionTimestamp'],
            event=event, 
            prePadding = -2.0,
            postPadding = -2.0
        )
        encoder = getEfdData(
            client,
            "lsst.sal.MTMount.encoder",
            event=event, 
            prePadding = -2.0,
            postPadding = -2.0
        )
        azValues = np.asarray(az["actualPosition"])
        elValues = np.asarray(el["actualPosition"])
        dAz = abs(azValues[0] - azValues[-1])
        dEl = abs(elValues[0] - elValues[-1])
        print(event.seqNum, dAz, dEl)
        if (dAz > 10.0) and (dEl > 10.0):
            break
    elPos = getEfdData(
        client,
        "lsst.sal.MTMount.logevent_elevationInPosition",
        event=event, 
        prePadding = -2.0,
        postPadding = -2.0
    )
    azPos = getEfdData(
        client,
        "lsst.sal.MTMount.logevent_azimuthInPosition",
        event=event, 
        prePadding = -2.0,
        postPadding = -2.0
    )
    
    azimuthT0 = encoder['azimuthEncoderPositionTimestamp0']
    azimuthT1 = encoder['azimuthEncoderPositionTimestamp1']
    azimuthT2 = encoder['azimuthEncoderPositionTimestamp2']
    azimuthT3 = encoder['azimuthEncoderPositionTimestamp3']
    azimuthdT1 = azimuthT1 - azimuthT0
    azimuthdT2 = azimuthT2 - azimuthT0
    azimuthdT3 = azimuthT3 - azimuthT0

    elevationT0 = encoder['elevationEncoderPositionTimestamp0']
    elevationT1 = encoder['elevationEncoderPositionTimestamp1']
    elevationT2 = encoder['elevationEncoderPositionTimestamp2']
    elevationT3 = encoder['elevationEncoderPositionTimestamp3']
    elevationdT1 = elevationT1 - elevationT0
    elevationdT2 = elevationT2 - elevationT0
    elevationdT3 = elevationT3 - elevationT0


    axs[0][n].set_title(f"{dayObs}, Slew event {event.seqNum}")
    axs[0][n].plot(azimuthdT1, label="TS1-TS0")
    axs[0][n].plot(azimuthdT2, label="TS2-TS0", ls='--')
    axs[0][n].plot(azimuthdT3, label="TS3-TS0", ls='dotted')
    axs[0][n].set_ylabel("Azimuth Encoder Timestamp deltas")
    axs[0][n].legend()
    
    axs[1][n].set_title(f"{dayObs}, Slew event {event.seqNum}")
    axs[1][n].plot(elevationdT1, label="TS1-TS0")
    axs[1][n].plot(elevationdT2, label="TS2-TS0", ls='--')
    axs[1][n].plot(elevationdT3, label="TS3-TS0", ls='dotted')
    axs[1][n].set_ylabel("ElevationEncoder Timestamp deltas")
    axs[1][n].legend()

fig.savefig("/home/c/cslage/u/MTMount/mount_plots/Encoder_Timestamp_Deltas_26Jun25.png")

# Re-coding to account for different encoder timestamps

In [ ]:
for plot in ['Offsets', 'Residuals']:
    fig, axs = plt.subplots(2, 3, figsize=(15, 10))
    plt.subplots_adjust(wspace = 0.5)
    for n, dayObs in enumerate([20250531, 20250601, 20250604]):
        events = eventMaker.getEvents(dayObs)
        myEvents = [e for e in events if ((e.type.name=='SLEWING') and (e.duration > 30.0))]
        print(f"for {dayObs}, there were {len(myEvents)} slews that were longer than 30 seconds")
    
        for event in myEvents:
            az = getEfdData(
                client,
                "lsst.sal.MTMount.azimuth",
                columns=['actualPosition', 'actualPositionTimestamp'],
                event=event, 
                prePadding = -2.0,
                postPadding = -2.0
            )
            el = getEfdData(
                client,
                "lsst.sal.MTMount.elevation",
                columns=['actualPosition', 'actualPositionTimestamp'],
                event=event, 
                prePadding = -2.0,
                postPadding = -2.0
            )
            encoder = getEfdData(
                client,
                "lsst.sal.MTMount.encoder",
                event=event, 
                prePadding = -2.0,
                postPadding = -2.0
            )
            azValues = np.asarray(az["actualPosition"])
            elValues = np.asarray(el["actualPosition"])
            dAz = abs(azValues[0] - azValues[-1])
            dEl = abs(elValues[0] - elValues[-1])
            print(event.seqNum, dAz, dEl)
            if (dAz > 10.0) and (dEl > 10.0):
                break
        elPos = getEfdData(
            client,
            "lsst.sal.MTMount.logevent_elevationInPosition",
            event=event, 
            prePadding = -2.0,
            postPadding = -2.0
        )
        azPos = getEfdData(
            client,
            "lsst.sal.MTMount.logevent_azimuthInPosition",
            event=event, 
            prePadding = -2.0,
            postPadding = -2.0
        )
        
        if len(azPos) > 0:
            azInPos = Time(azPos.index[0], scale='utc') - TimeDelta(2.0, format='sec')
            az = az[az['actualPositionTimestamp'] < azInPos.unix_tai] 
        azValues = np.asarray(az["actualPosition"])
        azValTimes = np.asarray(az["actualPositionTimestamp"])
        azEncInterp = np.zeros_like(azValues)
        for ii in range(4):
            azEncValsN = np.asarray(encoder[f'azimuthEncoderPosition{ii}'])
            azEncTimesN = np.asarray(encoder[f'azimuthEncoderPositionTimestamp{ii}'])
            azEncInterpN = np.interp(azValTimes, azEncTimesN, azEncValsN)
            azEncInterp += azEncInterpN
        azEncInterp /= 4.0      
        
        if len(elPos) > 0:
            elInPos = Time(elPos.index[0], scale='utc') - TimeDelta(2.0, format='sec')
            el = el[el['actualPositionTimestamp'] < elInPos.unix_tai] 
        elValues = np.asarray(el["actualPosition"])
        elValTimes = np.asarray(el["actualPositionTimestamp"])
        elEncInterp = np.zeros_like(elValues)
        for ii in range(4):
            elEncValsN = np.asarray(encoder[f'elevationEncoderPosition{ii}'])
            elEncTimesN = np.asarray(encoder[f'elevationEncoderPositionTimestamp{ii}'])
            elEncInterpN = np.interp(elValTimes, elEncTimesN, elEncValsN)
            elEncInterp += elEncInterpN
        elEncInterp /= 4.0      

        if plot == "Offsets":
            axs[0][n].set_title(f"{dayObs}, Slew event {event.seqNum}")
            axs[0][n].scatter(azValues, azEncInterp)
            azFit = np.polyfit(azValues, azEncInterp, 1)
            #print(fit)
            axs[0][n].text(0.1, 0.8, f"Slope = {azFit[0]:.6f}", transform=axs[0][n].transAxes)
            axs[0][n].text(0.1, 0.7, f"Offset = {(azFit[1]*3600):.2f} arcseconds", transform=axs[0][n].transAxes)
            axs[0][n].set_xlabel("Azimuth (degrees)")
            axs[0][n].set_ylabel("Azimuth Encoder average(degrees)")
            axs[0][n].set_title(f"{dayObs}, Slew event {event.seqNum}")
            axs[1][n].scatter(elValues, elEncInterp)
            elFit = np.polyfit(elValues, elEncInterp, 1)
            #print(fit)
            axs[1][n].text(0.1, 0.8, f"Slope = {elFit[0]:.6f}", transform=axs[1][n].transAxes)
            axs[1][n].text(0.1, 0.7, f"Offset = {(elFit[1]*3600):.2f} arcseconds", transform=axs[1][n].transAxes)
            axs[1][n].set_xlabel("Elevation (degrees)")
            axs[1][n].set_ylabel("Elevation Encoder average(degrees)")
        elif plot == "Residuals":
            axs[0][n].set_title(f"{dayObs}, Slew event {event.seqNum}")
            azFit = np.polyfit(azValues, azEncInterp, 1)
            print(azFit)
            azFitVals = np.polyval(azFit, azValues)
            axs[0][n].scatter(azValues, (azEncInterp - azFitVals) * 3600.0)
            axs[0][n].text(0.1, 0.8, f"Slope = {azFit[0]:.6f}", transform=axs[0][n].transAxes)
            axs[0][n].text(0.1, 0.7, f"Offset = {(azFit[1]*3600):.2f} arcseconds", transform=axs[0][n].transAxes)
            axs[0][n].set_xlabel("Azimuth (degrees)")
            axs[0][n].set_ylabel("Azimuth Encoder residuals(arcseconds)")
            axs[0][n].set_title(f"{dayObs}, Slew event {event.seqNum}")
            axs[0][n].set_ylim(-1000, 1000)
            elFit = np.polyfit(elValues, elEncInterp, 1)
            print(elFit)
            elFitVals = np.polyval(elFit, elValues)
            axs[1][n].scatter(elValues, (elEncInterp - elFitVals) * 3600.0)
            axs[1][n].text(0.1, 0.8, f"Slope = {elFit[0]:.6f}", transform=axs[1][n].transAxes)
            axs[1][n].text(0.1, 0.7, f"Offset = {(elFit[1]*3600):.2f} arcseconds", transform=axs[1][n].transAxes)
            axs[1][n].set_xlabel("Elevation (degrees)")
            axs[1][n].set_ylabel("Elevation Encoder residuals(arcseconds)")
            axs[1][n].set_ylim(-1000, 1000)
    
    fig.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Encoder_{plot}_Timestamps_26Jun25.png")

# Now look at tracking events

In [ ]:
for plot in ['Offsets', 'Residuals']:
    fig, axs = plt.subplots(2, 3, figsize=(15, 10))
    plt.subplots_adjust(wspace = 0.5)
    for n, dayObs in enumerate([20250531, 20250601, 20250604]):
        events = eventMaker.getEvents(dayObs)
        myEvents = [e for e in events if ((e.type.name=='TRACKING') and (e.duration > 30.0))]
        print(f"for {dayObs}, there were {len(myEvents)} tracks that were longer than 30 seconds")
    
        for event in myEvents:
            az = getEfdData(
                client,
                "lsst.sal.MTMount.azimuth",
                columns=['actualPosition', 'actualPositionTimestamp'],
                event=event, 
                prePadding = -2.0,
                postPadding = -2.0
            )
            el = getEfdData(
                client,
                "lsst.sal.MTMount.elevation",
                columns=['actualPosition', 'actualPositionTimestamp'],
                event=event, 
                prePadding = -2.0,
                postPadding = -2.0
            )
            encoder = getEfdData(
                client,
                "lsst.sal.MTMount.encoder",
                event=event, 
                prePadding = -2.0,
                postPadding = -2.0
            )
            azValues = np.asarray(az["actualPosition"])
            elValues = np.asarray(el["actualPosition"])
            dAz = abs(azValues[0] - azValues[-1])
            dEl = abs(elValues[0] - elValues[-1])
            print(event.seqNum, dAz, dEl)
            if (dAz > 1.0) and (dEl > 1.0):
                break
        print(event.seqNum, event.begin.isot)
        elPos = getEfdData(
            client,
            "lsst.sal.MTMount.logevent_elevationInPosition",
            event=event, 
            prePadding = -2.0,
            postPadding = -2.0
        )
        azPos = getEfdData(
            client,
            "lsst.sal.MTMount.logevent_azimuthInPosition",
            event=event, 
            prePadding = -2.0,
            postPadding = -2.0
        )
        
        if len(azPos) > 0:
            azInPos = Time(azPos.index[0], scale='utc') - TimeDelta(2.0, format='sec')
            az = az[az['actualPositionTimestamp'] < azInPos.unix_tai] 
        azValues = np.asarray(az["actualPosition"])
        azValTimes = np.asarray(az["actualPositionTimestamp"])
        azEncInterp = np.zeros_like(azValues)
        for ii in range(4):
            azEncValsN = np.asarray(encoder[f'azimuthEncoderPosition{ii}'])
            azEncTimesN = np.asarray(encoder[f'azimuthEncoderPositionTimestamp{ii}'])
            azEncInterpN = np.interp(azValTimes, azEncTimesN, azEncValsN)
            azEncInterp += azEncInterpN
        azEncInterp /= 4.0      
        
        if len(elPos) > 0:
            elInPos = Time(elPos.index[0], scale='utc') - TimeDelta(2.0, format='sec')
            el = el[el['actualPositionTimestamp'] < elInPos.unix_tai] 
        elValues = np.asarray(el["actualPosition"])
        elValTimes = np.asarray(el["actualPositionTimestamp"])
        elEncInterp = np.zeros_like(elValues)
        for ii in range(4):
            elEncValsN = np.asarray(encoder[f'elevationEncoderPosition{ii}'])
            elEncTimesN = np.asarray(encoder[f'elevationEncoderPositionTimestamp{ii}'])
            elEncInterpN = np.interp(elValTimes, elEncTimesN, elEncValsN)
            elEncInterp += elEncInterpN
        elEncInterp /= 4.0      

        if plot == "Offsets":
            axs[0][n].set_title(f"{dayObs}, Tracking event {event.seqNum}")
            axs[0][n].scatter(azValues, azEncInterp)
            azFit = np.polyfit(azValues, azEncInterp, 1)
            #print(fit)
            axs[0][n].text(0.1, 0.8, f"Slope = {azFit[0]:.6f}", transform=axs[0][n].transAxes)
            axs[0][n].text(0.1, 0.7, f"Offset = {(azFit[1]*3600):.2f} arcseconds", transform=axs[0][n].transAxes)
            axs[0][n].set_xlabel("Azimuth (degrees)")
            axs[0][n].set_ylabel("Azimuth Encoder average(degrees)")
            axs[0][n].set_title(f"{dayObs}, Tracking event {event.seqNum}")
            axs[1][n].scatter(elValues, elEncInterp)
            elFit = np.polyfit(elValues, elEncInterp, 1)
            #print(fit)
            axs[1][n].text(0.1, 0.8, f"Slope = {elFit[0]:.6f}", transform=axs[1][n].transAxes)
            axs[1][n].text(0.1, 0.7, f"Offset = {(elFit[1]*3600):.2f} arcseconds", transform=axs[1][n].transAxes)
            axs[1][n].set_xlabel("Elevation (degrees)")
            axs[1][n].set_ylabel("Elevation Encoder average(degrees)")
        elif plot == "Residuals":
            axs[0][n].set_title(f"{dayObs}, Tracking event {event.seqNum}")
            azFit = np.polyfit(azValues, azEncInterp, 1)
            print(azFit)
            azFitVals = np.polyval(azFit, azValues)
            axs[0][n].scatter(azValues, (azEncInterp - azFitVals) * 3600.0)
            axs[0][n].text(0.1, 0.8, f"Slope = {azFit[0]:.6f}", transform=axs[0][n].transAxes)
            axs[0][n].text(0.1, 0.7, f"Offset = {(azFit[1]*3600):.2f} arcseconds", transform=axs[0][n].transAxes)
            axs[0][n].set_xlabel("Azimuth (degrees)")
            axs[0][n].set_ylabel("Azimuth Encoder residuals(arcseconds)")
            axs[0][n].set_title(f"{dayObs}, Tracking event {event.seqNum}")
            axs[0][n].set_ylim(-10, 10)
            elFit = np.polyfit(elValues, elEncInterp, 1)
            print(elFit)
            elFitVals = np.polyval(elFit, elValues)
            axs[1][n].scatter(elValues, (elEncInterp - elFitVals) * 3600.0)
            axs[1][n].text(0.1, 0.8, f"Slope = {elFit[0]:.6f}", transform=axs[1][n].transAxes)
            axs[1][n].text(0.1, 0.7, f"Offset = {(elFit[1]*3600):.2f} arcseconds", transform=axs[1][n].transAxes)
            axs[1][n].set_xlabel("Elevation (degrees)")
            axs[1][n].set_ylabel("Elevation Encoder residuals(arcseconds)")
            axs[1][n].set_ylim(-10, 10)
    
    fig.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Encoder_{plot}_Tracking_26Jun25.png")

# Now look before and after homing

In [ ]:
homeDict[20250530]

In [ ]:
dayObs = 20250530
names = ['Before homing', 'After homing']
for m, [tStart, tEnd] in enumerate([[homeDict[20250530].index[1], homeDict[20250530].index[2]], [homeDict[20250530].index[2], Time("2025-05-31T09:12")]]):
    fig, axs = plt.subplots(2, 3, figsize=(15, 10))
    plt.subplots_adjust(wspace = 0.5)
    events = eventMaker.getEvents(dayObs)
    myEvents = [e for e in events if ((e.type.name=='TRACKING') and (e.duration > 30.0) and (e.begin > tStart) and (e.begin < tEnd))]
    print(f"for {dayObs}, there were {len(myEvents)} tracks that met the criteria")
    goodEvents = []
    for event in myEvents:
        az = getEfdData(
            client,
            "lsst.sal.MTMount.azimuth",
            columns=['actualPosition', 'actualPositionTimestamp'],
            event=event, 
            prePadding = -2.0,
            postPadding = -2.0
        )
        el = getEfdData(
            client,
            "lsst.sal.MTMount.elevation",
            columns=['actualPosition', 'actualPositionTimestamp'],
            event=event, 
            prePadding = -2.0,
            postPadding = -2.0
        )
        azValues = np.asarray(az["actualPosition"])
        elValues = np.asarray(el["actualPosition"])
        dAz = abs(azValues[0] - azValues[-1])
        dEl = abs(elValues[0] - elValues[-1])
        #print(event.seqNum, dAz, dEl)
        if (dAz > 0.6) and (dEl > 0.6):
            goodEvents.append(event)
    print(m, len(goodEvents))
    for n in range(3):
        event = goodEvents[n]
        az = getEfdData(
            client,
            "lsst.sal.MTMount.azimuth",
            columns=['actualPosition', 'actualPositionTimestamp'],
            event=event, 
            prePadding = -3.0,
            postPadding = -2.0
        )
        el = getEfdData(
            client,
            "lsst.sal.MTMount.elevation",
            columns=['actualPosition', 'actualPositionTimestamp'],
            event=event, 
            prePadding = -3.0,
            postPadding = -2.0
        )
        encoder = getEfdData(
            client,
            "lsst.sal.MTMount.encoder",
            event=event, 
            prePadding = -3.0,
            postPadding = -2.0
        )

        elPos = getEfdData(
            client,
            "lsst.sal.MTMount.logevent_elevationInPosition",
            event=event, 
            prePadding = -3.0,
            postPadding = -2.0
        )
        azPos = getEfdData(
            client,
            "lsst.sal.MTMount.logevent_azimuthInPosition",
            event=event, 
            prePadding = -3.0,
            postPadding = -2.0
        )
        
        if len(azPos) > 0:
            azInPos = Time(azPos.index[0], scale='utc') - TimeDelta(2.0, format='sec')
            az = az[az['actualPositionTimestamp'] < azInPos.unix_tai] 
        azValues = np.asarray(az["actualPosition"])
        azValTimes = np.asarray(az["actualPositionTimestamp"])
        azEncInterp = np.zeros_like(azValues)
        for ii in range(4):
            azEncValsN = np.asarray(encoder[f'azimuthEncoderPosition{ii}'])
            azEncTimesN = np.asarray(encoder[f'azimuthEncoderPositionTimestamp{ii}'])
            azEncInterpN = np.interp(azValTimes, azEncTimesN, azEncValsN)
            azEncInterp += azEncInterpN
        azEncInterp /= 4.0      
        
        if len(elPos) > 0:
            elInPos = Time(elPos.index[0], scale='utc') - TimeDelta(2.0, format='sec')
            el = el[el['actualPositionTimestamp'] < elInPos.unix_tai] 
        elValues = np.asarray(el["actualPosition"])
        elValTimes = np.asarray(el["actualPositionTimestamp"])
        elEncInterp = np.zeros_like(elValues)
        for ii in range(4):
            elEncValsN = np.asarray(encoder[f'elevationEncoderPosition{ii}'])
            elEncTimesN = np.asarray(encoder[f'elevationEncoderPositionTimestamp{ii}'])
            elEncInterpN = np.interp(elValTimes, elEncTimesN, elEncValsN)
            elEncInterp += elEncInterpN
        elEncInterp /= 4.0      
    
        axs[0][n].set_title(f"{dayObs}, Tracking event {event.seqNum} {names[m]}")
        axs[0][n].scatter(azValues, azEncInterp)
        azFit = np.polyfit(azValues, azEncInterp, 1)
        #print(fit)
        axs[0][n].text(0.1, 0.8, f"Slope = {azFit[0]:.6f}", transform=axs[0][n].transAxes)
        axs[0][n].text(0.1, 0.7, f"Offset = {(azFit[1]*3600):.2f} arcseconds", transform=axs[0][n].transAxes)
        axs[0][n].set_xlabel("Azimuth (degrees)")
        axs[0][n].set_ylabel("Azimuth Encoder average(degrees)")
        elFit = np.polyfit(elValues, elEncInterp, 1)
        #print(fit)
        axs[1][n].scatter(elValues, elEncInterp)
        axs[1][n].text(0.1, 0.8, f"Slope = {elFit[0]:.6f}", transform=axs[1][n].transAxes)
        axs[1][n].text(0.1, 0.7, f"Offset = {(elFit[1]*3600):.2f} arcseconds", transform=axs[1][n].transAxes)
        axs[1][n].set_xlabel("Elevation (degrees)")
        axs[1][n].set_ylabel("Elevation Encoder average(degrees)")

    fig.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Encoder_{names[m]}_Tracking_26Jun25.png")

# Now look at relative position.

In [ ]:
for plot in ['Offsets', 'Residuals']:
    fig, axs = plt.subplots(2, 3, figsize=(15, 10))
    plt.subplots_adjust(wspace = 0.5)
    for n, dayObs in enumerate([20250531, 20250601, 20250604]):
        events = eventMaker.getEvents(dayObs)
        myEvents = [e for e in events if ((e.type.name=='TRACKING') and (e.duration > 30.0))]
        #print(f"for {dayObs}, there were {len(myEvents)} tracks that were longer than 30 seconds")
    
        for event in myEvents:
            az = getEfdData(
                client,
                "lsst.sal.MTMount.azimuth",
                columns=['actualPosition', 'actualPositionTimestamp'],
                event=event, 
                prePadding = -2.0,
                postPadding = -2.0
            )
            el = getEfdData(
                client,
                "lsst.sal.MTMount.elevation",
                columns=['actualPosition', 'actualPositionTimestamp'],
                event=event, 
                prePadding = -2.0,
                postPadding = -2.0
            )
            encoder = getEfdData(
                client,
                "lsst.sal.MTMount.encoder",
                event=event, 
                prePadding = -2.0,
                postPadding = -2.0
            )
            azValues = np.asarray(az["actualPosition"])
            elValues = np.asarray(el["actualPosition"])
            dAz = abs(azValues[0] - azValues[-1])
            dEl = abs(elValues[0] - elValues[-1])
            #print(event.seqNum, dAz, dEl)
            if (dAz > 1.0) and (dEl > 1.0):
                break
        print(f" For{dayObs}, TMA event {event.seqNum}, start time is {event.begin.isot}, end time is {event.end.isot}")
        elPos = getEfdData(
            client,
            "lsst.sal.MTMount.logevent_elevationInPosition",
            event=event, 
            prePadding = -2.0,
            postPadding = -2.0
        )
        azPos = getEfdData(
            client,
            "lsst.sal.MTMount.logevent_azimuthInPosition",
            event=event, 
            prePadding = -2.0,
            postPadding = -2.0
        )
        
        if len(azPos) > 0:
            azInPos = Time(azPos.index[0], scale='utc') - TimeDelta(2.0, format='sec')
            az = az[az['actualPositionTimestamp'] < azInPos.unix_tai] 
        azValues = np.asarray(az["actualPosition"])
        azValTimes = np.asarray(az["actualPositionTimestamp"])
        azEncInterp = np.zeros_like(azValues)
        for ii in range(4):
            azEncValsN = np.asarray(encoder[f'azimuthEncoderRelativePosition{ii}'])
            azEncTimesN = np.asarray(encoder[f'azimuthEncoderRelativePositionTimestamp{ii}'])
            azEncInterpN = np.interp(azValTimes, azEncTimesN, azEncValsN)
            azEncInterp += azEncInterpN
        azEncInterp /= 4.0      
        
        if len(elPos) > 0:
            elInPos = Time(elPos.index[0], scale='utc') - TimeDelta(2.0, format='sec')
            el = el[el['actualPositionTimestamp'] < elInPos.unix_tai] 
        elValues = np.asarray(el["actualPosition"])
        elValTimes = np.asarray(el["actualPositionTimestamp"])
        elEncInterp = np.zeros_like(elValues)
        for ii in range(4):
            elEncValsN = np.asarray(encoder[f'elevationEncoderRelativePosition{ii}'])
            elEncTimesN = np.asarray(encoder[f'elevationEncoderRelativePositionTimestamp{ii}'])
            elEncInterpN = np.interp(elValTimes, elEncTimesN, elEncValsN)
            elEncInterp += elEncInterpN
        elEncInterp /= 4.0      

        if plot == "Offsets":
            axs[0][n].set_title(f"{dayObs}, Tracking event {event.seqNum}")
            axs[0][n].scatter(azValues, azEncInterp)
            azFit = np.polyfit(azValues, azEncInterp, 1)
            #print(fit)
            axs[0][n].text(0.1, 0.8, f"Slope = {azFit[0]:.6f}", transform=axs[0][n].transAxes)
            axs[0][n].text(0.1, 0.7, f"Offset = {(azFit[1]*3600):.2f} arcseconds", transform=axs[0][n].transAxes)
            axs[0][n].set_xlabel("Azimuth (degrees)")
            axs[0][n].set_ylabel("Azimuth Encoder average(degrees)")
            axs[0][n].set_title(f"{dayObs}, Tracking event {event.seqNum}")
            axs[1][n].scatter(elValues, elEncInterp)
            elFit = np.polyfit(elValues, elEncInterp, 1)
            #print(fit)
            axs[1][n].text(0.1, 0.8, f"Slope = {elFit[0]:.6f}", transform=axs[1][n].transAxes)
            axs[1][n].text(0.1, 0.7, f"Offset = {(elFit[1]*3600):.2f} arcseconds", transform=axs[1][n].transAxes)
            axs[1][n].set_xlabel("Elevation (degrees)")
            axs[1][n].set_ylabel("Elevation Encoder average(degrees)")
        elif plot == "Residuals":
            axs[0][n].set_title(f"{dayObs}, Tracking event {event.seqNum}")
            azFit = np.polyfit(azValues, azEncInterp, 1)
            print(azFit)
            azFitVals = np.polyval(azFit, azValues)
            axs[0][n].scatter(azValues, (azEncInterp - azFitVals) * 3600.0)
            axs[0][n].text(0.1, 0.8, f"Slope = {azFit[0]:.6f}", transform=axs[0][n].transAxes)
            axs[0][n].text(0.1, 0.7, f"Offset = {(azFit[1]*3600):.2f} arcseconds", transform=axs[0][n].transAxes)
            axs[0][n].set_xlabel("Azimuth (degrees)")
            axs[0][n].set_ylabel("Azimuth Encoder residuals(arcseconds)")
            axs[0][n].set_title(f"{dayObs}, Tracking event {event.seqNum}")
            axs[0][n].set_ylim(-10, 10)
            elFit = np.polyfit(elValues, elEncInterp, 1)
            print(elFit)
            elFitVals = np.polyval(elFit, elValues)
            axs[1][n].scatter(elValues, (elEncInterp - elFitVals) * 3600.0)
            axs[1][n].text(0.1, 0.8, f"Slope = {elFit[0]:.6f}", transform=axs[1][n].transAxes)
            axs[1][n].text(0.1, 0.7, f"Offset = {(elFit[1]*3600):.2f} arcseconds", transform=axs[1][n].transAxes)
            axs[1][n].set_xlabel("Elevation (degrees)")
            axs[1][n].set_ylabel("Elevation Encoder residuals(arcseconds)")
            axs[1][n].set_ylim(-10, 10)
    
    fig.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Encoder_{plot}_Relative_Tracking_26Jun25.png")

# Now look at 2025-06-29 when high speed telemetry was running

In [ ]:
dayObs = 20250629
for plot in ['Offsets', 'Residuals']:
    fig, axs = plt.subplots(2, 3, figsize=(15, 10))
    plt.subplots_adjust(wspace = 0.5)
    events = eventMaker.getEvents(dayObs)
    myEvents = [e for e in events if ((e.type.name=='TRACKING') and (e.duration > 30.0))]
    print(f"for {dayObs}, there were {len(myEvents)} tracks that met the criteria")
    goodEvents = []
    for event in myEvents:
        az = getEfdData(
            client,
            "lsst.sal.MTMount.azimuth",
            columns=['actualPosition', 'actualPositionTimestamp'],
            event=event, 
            prePadding = -2.0,
            postPadding = -2.0
        )
        el = getEfdData(
            client,
            "lsst.sal.MTMount.elevation",
            columns=['actualPosition', 'actualPositionTimestamp'],
            event=event, 
            prePadding = -2.0,
            postPadding = -2.0
        )
        azValues = np.asarray(az["actualPosition"])
        elValues = np.asarray(el["actualPosition"])
        dAz = abs(azValues[0] - azValues[-1])
        dEl = abs(elValues[0] - elValues[-1])
        #print(event.seqNum, dAz, dEl)
        if (dAz > 1.0) and (dEl > 1.0):
            goodEvents.append(event)
    print(len(goodEvents))
    
    for n in range(3):
        event = goodEvents[n]
        print(f" For{dayObs}, TMA event {event.seqNum}, start time is {event.begin.isot}, end time is {event.end.isot}")
        az = getEfdData(
            client,
            "lsst.sal.MTMount.azimuth",
            columns=['actualPosition', 'actualPositionTimestamp'],
            event=event, 
            prePadding = -3.0,
            postPadding = -2.0
        )
        el = getEfdData(
            client,
            "lsst.sal.MTMount.elevation",
            columns=['actualPosition', 'actualPositionTimestamp'],
            event=event, 
            prePadding = -3.0,
            postPadding = -2.0
        )
        encoder = getEfdData(
            client,
            "lsst.sal.MTMount.encoder",
            event=event, 
            prePadding = -3.0,
            postPadding = -2.0
        )
    
        elPos = getEfdData(
            client,
            "lsst.sal.MTMount.logevent_elevationInPosition",
            event=event, 
            prePadding = -3.0,
            postPadding = -2.0
        )
        azPos = getEfdData(
            client,
            "lsst.sal.MTMount.logevent_azimuthInPosition",
            event=event, 
            prePadding = -3.0,
            postPadding = -2.0
        )
        
        if len(azPos) > 0:
            azInPos = Time(azPos.index[0], scale='utc') - TimeDelta(2.0, format='sec')
            az = az[az['actualPositionTimestamp'] < azInPos.unix_tai] 
        azValues = np.asarray(az["actualPosition"])
        azValTimes = np.asarray(az["actualPositionTimestamp"])
        azEncInterp = np.zeros_like(azValues)
        for ii in range(4):
            azEncValsN = np.asarray(encoder[f'azimuthEncoderPosition{ii}'])
            azEncTimesN = np.asarray(encoder[f'azimuthEncoderPositionTimestamp{ii}'])
            azEncInterpN = np.interp(azValTimes, azEncTimesN, azEncValsN)
            azEncInterp += azEncInterpN
        azEncInterp /= 4.0      
        
        if len(elPos) > 0:
            elInPos = Time(elPos.index[0], scale='utc') - TimeDelta(2.0, format='sec')
            el = el[el['actualPositionTimestamp'] < elInPos.unix_tai] 
        elValues = np.asarray(el["actualPosition"])
        elValTimes = np.asarray(el["actualPositionTimestamp"])
        elEncInterp = np.zeros_like(elValues)
        for ii in range(4):
            elEncValsN = np.asarray(encoder[f'elevationEncoderPosition{ii}'])
            elEncTimesN = np.asarray(encoder[f'elevationEncoderPositionTimestamp{ii}'])
            elEncInterpN = np.interp(elValTimes, elEncTimesN, elEncValsN)
            elEncInterp += elEncInterpN
        elEncInterp /= 4.0      
    
        if plot == "Offsets":
            axs[0][n].set_title(f"{dayObs}, Tracking event {event.seqNum}")
            axs[0][n].scatter(azValues, azEncInterp)
            azFit = np.polyfit(azValues, azEncInterp, 1)
            #print(fit)
            axs[0][n].text(0.1, 0.8, f"Slope = {azFit[0]:.6f}", transform=axs[0][n].transAxes)
            axs[0][n].text(0.1, 0.7, f"Offset = {(azFit[1]*3600):.2f} arcseconds", transform=axs[0][n].transAxes)
            axs[0][n].set_xlabel("Azimuth (degrees)")
            axs[0][n].set_ylabel("Azimuth Encoder average(degrees)")
            axs[0][n].set_title(f"{dayObs}, Tracking event {event.seqNum}")
            axs[1][n].scatter(elValues, elEncInterp)
            elFit = np.polyfit(elValues, elEncInterp, 1)
            #print(fit)
            axs[1][n].text(0.1, 0.8, f"Slope = {elFit[0]:.6f}", transform=axs[1][n].transAxes)
            axs[1][n].text(0.1, 0.7, f"Offset = {(elFit[1]*3600):.2f} arcseconds", transform=axs[1][n].transAxes)
            axs[1][n].set_xlabel("Elevation (degrees)")
            axs[1][n].set_ylabel("Elevation Encoder average(degrees)")
        elif plot == "Residuals":
            axs[0][n].set_title(f"{dayObs}, Tracking event {event.seqNum}")
            azFit = np.polyfit(azValues, azEncInterp, 1)
            print(azFit)
            azFitVals = np.polyval(azFit, azValues)
            axs[0][n].scatter(azValues, (azEncInterp - azFitVals) * 3600.0)
            axs[0][n].text(0.1, 0.8, f"Slope = {azFit[0]:.6f}", transform=axs[0][n].transAxes)
            axs[0][n].text(0.1, 0.7, f"Offset = {(azFit[1]*3600):.2f} arcseconds", transform=axs[0][n].transAxes)
            axs[0][n].set_xlabel("Azimuth (degrees)")
            axs[0][n].set_ylabel("Azimuth Encoder residuals(arcseconds)")
            axs[0][n].set_title(f"{dayObs}, Tracking event {event.seqNum}")
            axs[0][n].set_ylim(-10, 10)
            elFit = np.polyfit(elValues, elEncInterp, 1)
            print(elFit)
            elFitVals = np.polyval(elFit, elValues)
            axs[1][n].scatter(elValues, (elEncInterp - elFitVals) * 3600.0)
            axs[1][n].text(0.1, 0.8, f"Slope = {elFit[0]:.6f}", transform=axs[1][n].transAxes)
            axs[1][n].text(0.1, 0.7, f"Offset = {(elFit[1]*3600):.2f} arcseconds", transform=axs[1][n].transAxes)
            axs[1][n].set_xlabel("Elevation (degrees)")
            axs[1][n].set_ylabel("Elevation Encoder residuals(arcseconds)")
            axs[1][n].set_ylim(-10, 10)
    
    fig.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Encoder_{plot}_HiSpeed_Telemetry_Tracking_30Jun25.png")

# Now look at how the offset varies with Az/El

In [ ]:
dayObs = 20250629
pdf = PdfPages(f"/home/c/cslage/u/MTMount/mount_plots/Offset_vs_AzEl{dayObs}.pdf")
fig = plt.figure(figsize=(8, 5))
events = eventMaker.getEvents(dayObs)
myEvents = [e for e in events if ((e.type.name=='TRACKING') and (e.duration > 29.0))]
print(f"for {dayObs}, there were {len(myEvents)} tracks that were longer than 29 seconds")
goodEvents = []
for event in myEvents:
    az = getEfdData(
        client,
        "lsst.sal.MTMount.azimuth",
        columns=['actualPosition', 'actualPositionTimestamp'],
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    el = getEfdData(
        client,
        "lsst.sal.MTMount.elevation",
        columns=['actualPosition', 'actualPositionTimestamp'],
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    azValues = np.asarray(az["actualPosition"])
    elValues = np.asarray(el["actualPosition"])
    dAz = abs(azValues[0] - azValues[-1])
    dEl = abs(elValues[0] - elValues[-1])
    #print(event.seqNum, dAz, dEl)
    if (dAz > 0.01) and (dEl > 0.01):
        goodEvents.append(event)
print(f"for {dayObs}, there were {len(goodEvents)} tracks that met the criteria")

azs = []
els = []
azOffsets = []
elOffsets = []
seqNums = []
for event in goodEvents:
    seqNums.append(event.seqNum)
    axs = []
    axs.append(fig.add_subplot(1,2,1))
    axs.append(fig.add_subplot(1,2,2))
    plt.subplots_adjust(wspace = 0.5)
    az = getEfdData(
        client,
        "lsst.sal.MTMount.azimuth",
        columns=['actualPosition', 'actualPositionTimestamp'],
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    el = getEfdData(
        client,
        "lsst.sal.MTMount.elevation",
        columns=['actualPosition', 'actualPositionTimestamp'],
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    encoder = getEfdData(
        client,
        "lsst.sal.MTMount.encoder",
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )

    elPos = getEfdData(
        client,
        "lsst.sal.MTMount.logevent_elevationInPosition",
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    azPos = getEfdData(
        client,
        "lsst.sal.MTMount.logevent_azimuthInPosition",
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    
    if len(azPos) > 0:
        azInPos = Time(azPos.index[0], scale='utc') - TimeDelta(2.0, format='sec')
        az = az[az['actualPositionTimestamp'] < azInPos.unix_tai] 
    azValues = np.asarray(az["actualPosition"])
    azs.append(azValues[0])
    azValTimes = np.asarray(az["actualPositionTimestamp"])
    azEncInterp = np.zeros_like(azValues)
    for ii in range(4):
        azEncValsN = np.asarray(encoder[f'azimuthEncoderPosition{ii}'])
        azEncTimesN = np.asarray(encoder[f'azimuthEncoderPositionTimestamp{ii}'])
        azEncInterpN = np.interp(azValTimes, azEncTimesN, azEncValsN)
        azEncInterp += azEncInterpN
    azEncInterp /= 4.0      
    
    if len(elPos) > 0:
        elInPos = Time(elPos.index[0], scale='utc') - TimeDelta(2.0, format='sec')
        el = el[el['actualPositionTimestamp'] < elInPos.unix_tai] 
    elValues = np.asarray(el["actualPosition"])
    els.append(elValues[0])
    elValTimes = np.asarray(el["actualPositionTimestamp"])
    elEncInterp = np.zeros_like(elValues)
    for ii in range(4):
        elEncValsN = np.asarray(encoder[f'elevationEncoderPosition{ii}'])
        elEncTimesN = np.asarray(encoder[f'elevationEncoderPositionTimestamp{ii}'])
        elEncInterpN = np.interp(elValTimes, elEncTimesN, elEncValsN)
        elEncInterp += elEncInterpN
    elEncInterp /= 4.0      

    axs[0].set_title(f"{dayObs}, Tracking event {event.seqNum}")
    axs[0].scatter(azValues, azEncInterp)
    azFit = np.polyfit(azValues, azEncInterp, 1)
    #print(fit)
    axs[0].text(0.1, 0.8, f"Slope = {azFit[0]:.6f}", transform=axs[0].transAxes)
    axs[0].text(0.1, 0.7, f"Offset = {(azFit[1]*3600):.2f} arcseconds", transform=axs[0].transAxes)
    azOffsets.append((azFit[1]*3600))
    axs[0].set_xlabel("Azimuth (degrees)")
    axs[0].set_ylabel("Azimuth Encoder average(degrees)")
    axs[0].set_title(f"{dayObs}, Tracking event {event.seqNum}")
    axs[1].scatter(elValues, elEncInterp)
    elFit = np.polyfit(elValues, elEncInterp, 1)
    #print(fit)
    axs[1].set_title(f"{dayObs}, Tracking event {event.seqNum}")
    axs[1].text(0.1, 0.8, f"Slope = {elFit[0]:.6f}", transform=axs[1].transAxes)
    axs[1].text(0.1, 0.7, f"Offset = {(elFit[1]*3600):.2f} arcseconds", transform=axs[1].transAxes)
    elOffsets.append((elFit[1]*3600))
    axs[1].set_xlabel("Elevation (degrees)")
    axs[1].set_ylabel("Elevation Encoder average(degrees)")
    pdf.savefig(fig)
    fig.clf()
pdf.close()  

fig = plt.figure(figsize=(8,5))
plt.subplots_adjust(wspace=0.5)
plt.suptitle(f"Encoder offsets {dayObs}")
plt.subplot(1,2,1)
plt.scatter(azs, azOffsets)
plt.ylim(-500,500)
plt.xlabel("Azimuth(degrees)")
plt.ylabel("Azimuth offset (arcseconds)")
plt.subplot(1,2,2)
plt.scatter(els, elOffsets)
plt.ylim(-800,800)
plt.xlabel("Elevation(degrees)")
plt.ylabel("Elevation offset (arcseconds)")
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Encoder_Offset_Summary_{dayObs}.png")

# Repeat this with two changes:
## Only include up to the first homing
## Plot residuals on each plot

In [ ]:
dayObs = 20250629
homes = homeDict[dayObs]
pdf = PdfPages(f"/home/c/cslage/u/MTMount/mount_plots/Offset_vs_AzEl{dayObs}.pdf")
fig = plt.figure(figsize=(8, 5))
events = eventMaker.getEvents(dayObs)
myEvents = [e for e in events if ((e.type.name=='TRACKING') and (e.duration > 29.0) and (e.begin > homeDict[dayObs].index[1]))]
print(f"for {dayObs}, there were {len(myEvents)} tracks that were longer than 29 seconds")
goodEvents = []
for event in myEvents:
    az = getEfdData(
        client,
        "lsst.sal.MTMount.azimuth",
        columns=['actualPosition', 'actualPositionTimestamp'],
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    el = getEfdData(
        client,
        "lsst.sal.MTMount.elevation",
        columns=['actualPosition', 'actualPositionTimestamp'],
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    azValues = np.asarray(az["actualPosition"])
    elValues = np.asarray(el["actualPosition"])
    dAz = abs(azValues[0] - azValues[-1])
    dEl = abs(elValues[0] - elValues[-1])
    #print(event.seqNum, dAz, dEl)
    if (dAz > 0.01) and (dEl > 0.01):
        goodEvents.append(event)
print(f"for {dayObs}, there were {len(goodEvents)} tracks that met the criteria")

azs = []
els = []
azOffsets = []
elOffsets = []
seqNums = []
for event in goodEvents[::4]:
    seqNums.append(event.seqNum)
    axs = []
    axs.append(fig.add_subplot(2,2,1))
    axs.append(fig.add_subplot(2,2,2))
    axs.append(fig.add_subplot(2,2,3))
    axs.append(fig.add_subplot(2,2,4))
    plt.subplots_adjust(wspace=0.5, hspace=0.5)
    az = getEfdData(
        client,
        "lsst.sal.MTMount.azimuth",
        columns=['actualPosition', 'actualPositionTimestamp'],
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    el = getEfdData(
        client,
        "lsst.sal.MTMount.elevation",
        columns=['actualPosition', 'actualPositionTimestamp'],
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    encoder = getEfdData(
        client,
        "lsst.sal.MTMount.encoder",
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )

    elPos = getEfdData(
        client,
        "lsst.sal.MTMount.logevent_elevationInPosition",
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    azPos = getEfdData(
        client,
        "lsst.sal.MTMount.logevent_azimuthInPosition",
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    
    if len(azPos) > 0:
        azInPos = Time(azPos.index[0], scale='utc') - TimeDelta(2.0, format='sec')
        az = az[az['actualPositionTimestamp'] < azInPos.unix_tai] 
    azValues = np.asarray(az["actualPosition"])
    azs.append(azValues[0])
    azValTimes = np.asarray(az["actualPositionTimestamp"])
    azEncInterp = np.zeros_like(azValues)
    for ii in range(4):
        azEncValsN = np.asarray(encoder[f'azimuthEncoderPosition{ii}'])
        azEncTimesN = np.asarray(encoder[f'azimuthEncoderPositionTimestamp{ii}'])
        azEncInterpN = np.interp(azValTimes, azEncTimesN, azEncValsN)
        azEncInterp += azEncInterpN
    azEncInterp /= 4.0      
    
    if len(elPos) > 0:
        elInPos = Time(elPos.index[0], scale='utc') - TimeDelta(2.0, format='sec')
        el = el[el['actualPositionTimestamp'] < elInPos.unix_tai] 
    elValues = np.asarray(el["actualPosition"])
    els.append(elValues[0])
    elValTimes = np.asarray(el["actualPositionTimestamp"])
    elEncInterp = np.zeros_like(elValues)
    for ii in range(4):
        elEncValsN = np.asarray(encoder[f'elevationEncoderPosition{ii}'])
        elEncTimesN = np.asarray(encoder[f'elevationEncoderPositionTimestamp{ii}'])
        elEncInterpN = np.interp(elValTimes, elEncTimesN, elEncValsN)
        elEncInterp += elEncInterpN
    elEncInterp /= 4.0      

    axs[0].set_title(f"{dayObs}, Tracking event {event.seqNum}")
    axs[0].scatter(azValues, azEncInterp)
    azFit = np.polyfit(azValues, azEncInterp, 1)
    #print(fit)
    axs[0].text(0.1, 0.8, f"Slope = {azFit[0]:.6f}", transform=axs[0].transAxes)
    axs[0].text(0.1, 0.7, f"Offset = {(azFit[1]*3600):.2f} arcseconds", transform=axs[0].transAxes)
    azOffsets.append((azFit[1]*3600))
    axs[0].set_xlabel("Azimuth (degrees)")
    axs[0].set_ylabel("Az Encoder average(degrees)")
    axs[0].set_title(f"{dayObs}, Tracking event {event.seqNum}")
    axs[1].scatter(elValues, elEncInterp)
    elFit = np.polyfit(elValues, elEncInterp, 1)
    #print(fit)
    axs[1].set_title(f"{dayObs}, Tracking event {event.seqNum}")
    axs[1].text(0.1, 0.8, f"Slope = {elFit[0]:.6f}", transform=axs[1].transAxes)
    axs[1].text(0.1, 0.7, f"Offset = {(elFit[1]*3600):.2f} arcseconds", transform=axs[1].transAxes)
    elOffsets.append((elFit[1]*3600))
    axs[1].set_xlabel("Elevation (degrees)")
    axs[1].set_ylabel("El Encoder average(degrees)")
    azFit = np.polyfit(azValues, azEncInterp, 1)
    print(azFit)
    azFitVals = np.polyval(azFit, azValues)
    axs[2].scatter(azValues, (azEncInterp - azFitVals) * 3600.0)
    axs[2].text(0.1, 0.8, f"Slope = {azFit[0]:.6f}", transform=axs[2].transAxes)
    axs[2].text(0.1, 0.7, f"Offset = {(azFit[1]*3600):.2f} arcseconds", transform=axs[2].transAxes)
    axs[2].set_xlabel("Azimuth (degrees)")
    axs[2].set_ylabel("Az Encoder residuals(arcseconds)")
    axs[2].set_title(f"{dayObs}, Tracking event {event.seqNum}")
    axs[2].set_ylim(-10, 10)
    elFit = np.polyfit(elValues, elEncInterp, 1)
    print(elFit)
    elFitVals = np.polyval(elFit, elValues)
    axs[3].scatter(elValues, (elEncInterp - elFitVals) * 3600.0)
    axs[3].text(0.1, 0.8, f"Slope = {elFit[0]:.6f}", transform=axs[3].transAxes)
    axs[3].text(0.1, 0.7, f"Offset = {(elFit[1]*3600):.2f} arcseconds", transform=axs[3].transAxes)
    axs[3].set_xlabel("Elevation (degrees)")
    axs[3].set_ylabel("El Encoder residuals(arcseconds)")
    axs[3].set_ylim(-10, 10)
    pdf.savefig(fig)
    fig.clf()
pdf.close()  

fig = plt.figure(figsize=(8,5))
plt.subplots_adjust(wspace=0.5)
plt.suptitle(f"Encoder offsets {dayObs}")
plt.subplot(1,2,1)
plt.scatter(azs, azOffsets)
plt.ylim(-500,500)
plt.xlabel("Azimuth(degrees)")
plt.ylabel("Azimuth offset (arcseconds)")
plt.subplot(1,2,2)
plt.scatter(els, elOffsets)
plt.ylim(-800,800)
plt.xlabel("Elevation(degrees)")
plt.ylabel("Elevation offset (arcseconds)")
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Encoder_Offset_Summary_{dayObs}.png")

# Repeat again without the plots

In [ ]:
dayObs = 20250629
homes = homeDict[dayObs]
events = eventMaker.getEvents(dayObs)
myEvents = [e for e in events if ((e.type.name=='TRACKING') and (e.duration > 29.0) and (e.begin > homeDict[dayObs].index[1]))]
print(f"for {dayObs}, there were {len(myEvents)} tracks that were longer than 29 seconds")
goodEvents = []
for event in myEvents:
    az = getEfdData(
        client,
        "lsst.sal.MTMount.azimuth",
        columns=['actualPosition', 'actualPositionTimestamp'],
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    el = getEfdData(
        client,
        "lsst.sal.MTMount.elevation",
        columns=['actualPosition', 'actualPositionTimestamp'],
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    azValues = np.asarray(az["actualPosition"])
    elValues = np.asarray(el["actualPosition"])
    dAz = abs(azValues[0] - azValues[-1])
    dEl = abs(elValues[0] - elValues[-1])
    print(event.seqNum, dAz, dEl)
    if (dAz > 0.01) and (dEl > 0.01):
        goodEvents.append(event)
print(f"for {dayObs}, there were {len(goodEvents)} tracks that met the criteria")

offsetDict = {}
for event in goodEvents:
    az = getEfdData(
        client,
        "lsst.sal.MTMount.azimuth",
        columns=['actualPosition', 'actualPositionTimestamp'],
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    el = getEfdData(
        client,
        "lsst.sal.MTMount.elevation",
        columns=['actualPosition', 'actualPositionTimestamp'],
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    encoder = getEfdData(
        client,
        "lsst.sal.MTMount.encoder",
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )

    elPos = getEfdData(
        client,
        "lsst.sal.MTMount.logevent_elevationInPosition",
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    azPos = getEfdData(
        client,
        "lsst.sal.MTMount.logevent_azimuthInPosition",
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    
    if len(azPos) > 0:
        azInPos = Time(azPos.index[0], scale='utc') - TimeDelta(2.0, format='sec')
        az = az[az['actualPositionTimestamp'] < azInPos.unix_tai] 
    azValues = np.asarray(az["actualPosition"])
    azValTimes = np.asarray(az["actualPositionTimestamp"])
    azEncInterp = np.zeros_like(azValues)
    for ii in range(4):
        azEncValsN = np.asarray(encoder[f'azimuthEncoderPosition{ii}'])
        azEncTimesN = np.asarray(encoder[f'azimuthEncoderPositionTimestamp{ii}'])
        azEncInterpN = np.interp(azValTimes, azEncTimesN, azEncValsN)
        azEncInterp += azEncInterpN
    azEncInterp /= 4.0      
    
    if len(elPos) > 0:
        elInPos = Time(elPos.index[0], scale='utc') - TimeDelta(2.0, format='sec')
        el = el[el['actualPositionTimestamp'] < elInPos.unix_tai] 
    elValues = np.asarray(el["actualPosition"])
    elValTimes = np.asarray(el["actualPositionTimestamp"])
    elEncInterp = np.zeros_like(elValues)
    for ii in range(4):
        elEncValsN = np.asarray(encoder[f'elevationEncoderPosition{ii}'])
        elEncTimesN = np.asarray(encoder[f'elevationEncoderPositionTimestamp{ii}'])
        elEncInterpN = np.interp(elValTimes, elEncTimesN, elEncValsN)
        elEncInterp += elEncInterpN
    elEncInterp /= 4.0      

    azFit = np.polyfit(azValues, azEncInterp, 1)
    azOffset = azFit[1]*3600.0
    elFit = np.polyfit(elValues, elEncInterp, 1)
    elOffset = elFit[1]*3600
    offsetDict[event.seqNum] = [azValues[0], elValues[0], azOffset, elOffset]
    print(f"Finished seqNum {event.seqNum}")
filename = f"/home/c/cslage/u/MTMount/mount_plots/OffsetDict_{dayObs}.pkl"
with open(filename, 'wb') as f:
    pkl.dump(offsetDict, f)


In [ ]:
azs = []
els = []
azOffsets = []
elOffsets = []
for seqNum in offsetDict.keys():
    [az, el, azOffset, elOffset] = offsetDict[seqNum]
    azs.append(az)
    els.append(el)
    azOffsets.append(azOffset)
    elOffsets.append(elOffset)

fig = plt.figure(figsize=(8,5))
plt.subplots_adjust(wspace=0.5)
plt.suptitle(f"Encoder offsets {dayObs}")
plt.subplot(1,2,1)
plt.scatter(azs, azOffsets)
plt.ylim(-500,500)
plt.xlabel("Azimuth(degrees)")
plt.ylabel("Azimuth offset (arcseconds)")
plt.subplot(1,2,2)
plt.scatter(els, elOffsets)
plt.ylim(-800,800)
plt.xlabel("Elevation(degrees)")
plt.ylabel("Elevation offset (arcseconds)")
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Encoder_Offset_Summary_{dayObs}.png")

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.scatter(np.array(azs) * np.pi / 180.0, 90.0 - np.array(els), marker = 'x')
ax.set_rmax(60.0)
r_values = [10.0, 20.0, 30.0, 40.0, 50.0]
r_labels = [80.0, 70.0, 60.0, 50.0, 40.0]
ax.set_rgrids(r_values, r_labels)
ax.grid(True)


In [ ]:
dayObs = 20250629
homes = homeDict[dayObs]
events = eventMaker.getEvents(dayObs)
myEvents = [e for e in events if ((e.type.name=='TRACKING') and (e.duration > 29.0) and (e.begin > homeDict[dayObs].index[1]))]
print(f"for {dayObs}, there were {len(myEvents)} tracks that were longer than 29 seconds")
goodEvents = []
for event in myEvents:
    az = getEfdData(
        client,
        "lsst.sal.MTMount.azimuth",
        columns=['actualPosition', 'actualPositionTimestamp'],
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    el = getEfdData(
        client,
        "lsst.sal.MTMount.elevation",
        columns=['actualPosition', 'actualPositionTimestamp'],
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    azValues = np.asarray(az["actualPosition"])
    elValues = np.asarray(el["actualPosition"])
    dAz = abs(azValues[0] - azValues[-1])
    dEl = abs(elValues[0] - elValues[-1])
    print(event.seqNum, dAz, dEl)
    if (dAz > 0.01) and (dEl > 0.01):
        goodEvents.append(event)
print(f"for {dayObs}, there were {len(goodEvents)} tracks that met the criteria")



In [ ]:

for event in goodEvents[3:4]:
    seqNum = event.seqNum
    az = getEfdData(
        client,
        "lsst.sal.MTMount.azimuth",
        columns=['actualPosition', 'actualPositionTimestamp'],
        event=event, 
        prePadding = 20.0,
        postPadding = 20.0
    )
    el = getEfdData(
        client,
        "lsst.sal.MTMount.elevation",
        columns=['actualPosition', 'actualPositionTimestamp'],
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    encoder = getEfdData(
        client,
        "lsst.sal.MTMount.encoder",
        event=event, 
        prePadding = 20.0,
        postPadding = 20.0
    )
    azValues = np.asarray(az["actualPosition"])
    azValTimes = np.asarray(az["actualPositionTimestamp"])
    azEncInterp = np.zeros_like(azValues)
    for ii in range(4):
        azEncValsN = np.asarray(encoder[f'azimuthEncoderPosition{ii}'])
        azEncTimesN = np.asarray(encoder[f'azimuthEncoderPositionTimestamp{ii}'])
        azEncInterpN = np.interp(azValTimes, azEncTimesN, azEncValsN)
        azEncInterp += azEncInterpN
    azEncInterp /= 4.0
    #azValues = azValues[0:-(azDelay+1)]
    #azEncInterp = azEncInterp[azDelay:-1]
    
    elValues = np.asarray(el["actualPosition"])
    elValTimes = np.asarray(el["actualPositionTimestamp"])
    elEncInterp = np.zeros_like(elValues)
    for ii in range(4):
        elEncValsN = np.asarray(encoder[f'elevationEncoderPosition{ii}'])
        elEncTimesN = np.asarray(encoder[f'elevationEncoderPositionTimestamp{ii}'])
        elEncInterpN = np.interp(elValTimes, elEncTimesN, elEncValsN)
        elEncInterp += elEncInterpN
    elEncInterp /= 4.0      

    azFit = np.polyfit(azValues, azEncInterp, 1)
    azOffset = azFit[1]*3600.0
    elFit = np.polyfit(elValues, elEncInterp, 1)
    elOffset = elFit[1]*3600
    print(seqNum, azOffset, elOffset)


In [ ]:
from scipy.optimize import minimize
maxDeltaT = 5000E-3
def calcDeltaT(params, args):
    # This calculates the deltaT needed
    # to minimize the offset
    [values, valTimes, demand, demTimes] = args
    [deltaT] = params
    demandInterp = np.interp(valTimes, demTimes + deltaT, demand)
    error = (values - demandInterp)**2 * 3600
    value = np.median(error)
    return value


# Calculate the deltaT needed to drive the median(error) to zero
ii = 0
azEncVals0 = np.asarray(encoder[f'azimuthEncoderPosition{ii}'])
azEncTimes0 = np.asarray(encoder[f'azimuthEncoderPositionTimestamp{ii}'])

args = [azValues, azValTimes, azEncVals0, azEncTimes0]
x0 = [0.0]
result = minimize(calcDeltaT, x0, args=args, method="Powell", bounds=[(-maxDeltaT, maxDeltaT)])
print(result)
deltaTAz = -1.0#result.x[0]

azEncInterp = np.interp(azValTimes, azEncTimes0 + deltaTAz, azEncVals0)

In [ ]:
# This is event 7
fig, axs = plt.subplots(2,1,figsize = (12,5))
plt.subplots_adjust(hspace=0.5)
plt.suptitle(f"Tracking event {event.seqNum} {(event.begin - TimeDelta(20.0, format='sec')).isot} - {(event.end + TimeDelta(20.0, format='sec')).isot}")                                                       
ii = 0
azEncVals0 = np.asarray(encoder[f'azimuthEncoderPosition{ii}'])
azEncTimes0 = np.asarray(encoder[f'azimuthEncoderPositionTimestamp{ii}'])
axs[0].set_title("No time delay")
axs[0].plot(azValTimes, azValues, label = 'actual position vs actualPositionTimestamp')
axs[0].plot(azEncTimes0, azEncVals0, ls='--', label='azimuthEncoderPosition0 vs azimuthEncoderPositionTimestamp0')
axs[0].legend(loc='lower right')
axs[1].set_title("-1.0 second time delay")
axs[1].plot(azValTimes, azValues, label = 'actual position vs actualPositionTimestamp')
axs[1].plot(azEncTimes0 - 1.0, azEncVals0, ls='--', label='azimuthEncoderPosition0 vs azimuthEncoderPositionTimestamp0')
axs[1].legend(loc='lower right')
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Encoder_Time_Delay_{dayObs}.png")

In [ ]:

for event in goodEvents[31:32]:
    seqNum = event.seqNum
    az = getEfdData(
        client,
        "lsst.sal.MTMount.azimuth",
        columns=['actualPosition', 'actualPositionTimestamp'],
        event=event, 
        prePadding = 20.0,
        postPadding = 20.0
    )
    el = getEfdData(
        client,
        "lsst.sal.MTMount.elevation",
        columns=['actualPosition', 'actualPositionTimestamp'],
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    encoder = getEfdData(
        client,
        "lsst.sal.MTMount.encoder",
        event=event, 
        prePadding = 20.0,
        postPadding = 20.0
    )
    azValues = np.asarray(az["actualPosition"])
    azValTimes = np.asarray(az["actualPositionTimestamp"])
    azValTimes -= azValTimes[0]
    ii = 0
    azEncVals0 = np.asarray(encoder[f'azimuthEncoderPosition{ii}'])
    azEncTimes0 = np.asarray(encoder[f'azimuthEncoderPositionTimestamp{ii}'])
    azEncTimes0 -= azEncTimes0[0]


In [ ]:
tBegin = 5.0
tEnd = azValTimes[-1] - 5.0
nSteps = int((tEnd - tBegin)) * 50 + 1
refTimes = np.linspace(tBegin, tEnd, nSteps)
#azTimes[0:10]

from scipy.optimize import minimize
maxDeltaT = 2.0
def calcDeltaT(params, args):
    # This calculates the deltaT needed
    # to minimize the offset
    [values, valTimes, encVals, encTimes, refTimes] = args
    [deltaT] = params
    valsInterp = np.interp(refTimes, valTimes, values)
    encInterp = np.interp(refTimes, encTimes - deltaT, encVals)
    error = np.sum((valsInterp - encInterp) * (valsInterp - encInterp))
    return error


# Calculate the deltaT needed to drive the median(error) to zero

args = [azValues, azValTimes, azEncVals0, azEncTimes0, refTimes]
x0 = [0.0]
result = minimize(calcDeltaT, x0, args=args, method="Powell", bounds=[(-maxDeltaT, maxDeltaT)])
print(result.x[0])


In [ ]:
# This is event 3
fig, axs = plt.subplots(2,1,figsize = (12,5))
plt.subplots_adjust(hspace=0.5)
plt.suptitle(f"Tracking event {event.seqNum} {(event.begin - TimeDelta(20.0, format='sec')).isot} - {(event.end + TimeDelta(20.0, format='sec')).isot}")                                                       
valsInterp = np.interp(refTimes, azValTimes, azValues)
encInterp = np.interp(refTimes, azEncTimes0, azEncVals0)
axs[0].set_title("No time delay")
axs[0].plot(refTimes, valsInterp, label = 'actual position vs actualPositionTimestamp')
axs[0].plot(refTimes, encInterp, ls='--', label='azimuthEncoderPosition0 vs azimuthEncoderPositionTimestamp0')
axs[0].legend(loc='lower right')
valsInterp = np.interp(refTimes, azValTimes, azValues)
encInterp = np.interp(refTimes, azEncTimes0 - result.x[0], azEncVals0)
axs[1].set_title(f"{result.x[0]:.4f} second time delay")
axs[1].plot(refTimes, valsInterp, label = 'actual position vs actualPositionTimestamp')
axs[1].plot(refTimes, encInterp, ls='--', label='azimuthEncoderPosition0 vs azimuthEncoderPositionTimestamp0')
axs[1].legend(loc='lower right')
#plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Encoder_Time_Delay_{dayObs}.png")

# Now calculate the delays for all of the tracking events

In [ ]:
from scipy.optimize import minimize
maxDeltaT = 2.0
def calcDeltaT(params, args):
    # This calculates the deltaT needed
    # to minimize the offset
    [values, valTimes, encVals, encTimes, refTimes] = args
    [deltaT] = params
    valsInterp = np.interp(refTimes, valTimes, values)
    encInterp = np.interp(refTimes, encTimes - deltaT, encVals)
    error = np.sum((valsInterp - encInterp) * (valsInterp - encInterp))
    return error


dayObs = 20250629
pdf = PdfPages(f"/home/c/cslage/u/MTMount/mount_plots/Encoder_Delays_{dayObs}.pdf")
fig = plt.figure(figsize=(12, 10))

homes = homeDict[dayObs]
events = eventMaker.getEvents(dayObs)
myEvents = [e for e in events if ((e.type.name=='TRACKING') and (e.duration > 29.0) and (e.begin > homeDict[dayObs].index[1]))]
print(f"for {dayObs}, there were {len(myEvents)} tracks that were longer than 29 seconds")
goodEvents = []
for event in myEvents:
    az = getEfdData(
        client,
        "lsst.sal.MTMount.azimuth",
        columns=['actualPosition', 'actualPositionTimestamp'],
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    el = getEfdData(
        client,
        "lsst.sal.MTMount.elevation",
        columns=['actualPosition', 'actualPositionTimestamp'],
        event=event, 
        prePadding = -1.0,
        postPadding = -1.0
    )
    azValues = np.asarray(az["actualPosition"])
    elValues = np.asarray(el["actualPosition"])
    dAz = abs(azValues[0] - azValues[-1])
    dEl = abs(elValues[0] - elValues[-1])
    #print(event.seqNum, dAz, dEl)
    if (dAz > 0.01) and (dEl > 0.01):
        goodEvents.append(event)
print(f"for {dayObs}, there were {len(goodEvents)} tracks that met the criteria")
azDelays = {}
elDelays = {}
for ii in range(4): # Encoder head
    for event in goodEvents:
        seqNum = event.seqNum
        axs = []
        axs.append(fig.add_subplot(4,1,1))
        axs.append(fig.add_subplot(4,1,2))
        axs.append(fig.add_subplot(4,1,3))
        axs.append(fig.add_subplot(4,1,4))
        plt.subplots_adjust(hspace=0.5)

        az = getEfdData(
            client,
            "lsst.sal.MTMount.azimuth",
            columns=['actualPosition', 'actualPositionTimestamp'],
            event=event, 
            prePadding = 20.0,
            postPadding = 20.0
        )
        el = getEfdData(
            client,
            "lsst.sal.MTMount.elevation",
            columns=['actualPosition', 'actualPositionTimestamp'],
            event=event, 
            prePadding = 20.0,
            postPadding = 20.0
        )
        encoder = getEfdData(
            client,
            "lsst.sal.MTMount.encoder",
            event=event, 
            prePadding = 20.0,
            postPadding = 20.0
        )
        azValues = np.asarray(az["actualPosition"])
        azValTimes = np.asarray(az["actualPositionTimestamp"])
        azValTimes -= azValTimes[0]
        azEncVals = np.asarray(encoder[f'azimuthEncoderPosition{ii}'])
        azEncTimes = np.asarray(encoder[f'azimuthEncoderPositionTimestamp{ii}'])
        azEncTimes -= azEncTimes[0]

        tBegin = 5.0
        tEnd = azValTimes[-1] - 5.0
        nSteps = int((tEnd - tBegin)) * 50 + 1
        azRefTimes = np.linspace(tBegin, tEnd, nSteps)
        args = [azValues, azValTimes, azEncVals, azEncTimes, azRefTimes]
        x0 = [0.0]
        result = minimize(calcDeltaT, x0, args=args, method="Powell", bounds=[(-maxDeltaT, maxDeltaT)])
        azDelay = result.x[0]
        azDelays[f"{seqNum}_{ii}"] = azDelay

        elValues = np.asarray(el["actualPosition"])
        elValTimes = np.asarray(el["actualPositionTimestamp"])
        elValTimes -= elValTimes[0]
        elEncVals = np.asarray(encoder[f'elevationEncoderPosition{ii}'])
        elEncTimes = np.asarray(encoder[f'elevationEncoderPositionTimestamp{ii}'])
        elEncTimes -= elEncTimes[0]

        tBegin = 5.0
        tEnd = elValTimes[-1] - 5.0
        nSteps = int((tEnd - tBegin)) * 50 + 1
        elRefTimes = np.linspace(tBegin, tEnd, nSteps)
        args = [elValues, elValTimes, elEncVals, elEncTimes, elRefTimes]
        x0 = [0.0]
        result = minimize(calcDeltaT, x0, args=args, method="Powell", bounds=[(-maxDeltaT, maxDeltaT)])
        elDelay = result.x[0]
        elDelays[f"{seqNum}_{ii}"] = elDelay


        plt.suptitle(f"Tracking event {event.seqNum} {(event.begin - TimeDelta(15.0, format='sec')).isot} - {(event.end + TimeDelta(15.0, format='sec')).isot}")                                                       
        valsInterp = np.interp(azRefTimes, azValTimes, azValues)
        encInterp = np.interp(azRefTimes, azEncTimes, azEncVals)
        axs[0].set_title("Az - no time delay")
        axs[0].plot(azRefTimes, valsInterp, label = 'actual position vs actualPositionTimestamp')
        axs[0].plot(azRefTimes, encInterp, ls='--', label=f'encoderPosition{ii} vs encoderPositionTimestamp{ii}')
        axs[0].legend()
        valsInterp = np.interp(azRefTimes, azValTimes, azValues)
        encInterp = np.interp(azRefTimes, azEncTimes - azDelay, azEncVals)
        axs[1].set_title(f"Az - {azDelay:.4f} second time delay")
        axs[1].plot(azRefTimes, valsInterp, label = 'actual position vs actualPositionTimestamp')
        axs[1].plot(azRefTimes, encInterp, ls='--', label=f'encoderPosition{ii} vs encoderPositionTimestamp{ii}')
        axs[1].legend()
        valsInterp = np.interp(elRefTimes, elValTimes, elValues)
        encInterp = np.interp(elRefTimes, elEncTimes, elEncVals)
        axs[2].set_title("El - no time delay")
        axs[2].plot(elRefTimes, valsInterp, label = 'actual position vs actualPositionTimestamp')
        axs[2].plot(elRefTimes, encInterp, ls='--', label=f'encoderPosition{ii} vs encoderPositionTimestamp{ii}')
        axs[2].legend()
        valsInterp = np.interp(elRefTimes, elValTimes, elValues)
        encInterp = np.interp(elRefTimes, elEncTimes - elDelay, elEncVals)
        axs[3].set_title(f"El - {elDelay:.4f} second time delay")
        axs[3].plot(elRefTimes, valsInterp, label = 'actual position vs actualPositionTimestamp')
        axs[3].plot(elRefTimes, encInterp, ls='--', label=f'encoderPosition{ii} vs encoderPositionTimestamp{ii}')
        axs[3].legend()
        pdf.savefig(fig)
        fig.clf()
        print(f"Finished seqNum {seqNum}, encoder head {ii}.")
pdf.close()  


In [ ]:
filename = f"/home/c/cslage/u/MTMount/mount_plots/Encoder_Delay_Dict_{dayObs}.pkl"
with open(filename, 'wb') as f:
    pkl.dump([azDelays, elDelays], f)


In [ ]:
filename = f"/home/c/cslage/u/MTMount/mount_plots/Encoder_Delay_Dict_{dayObs}.pkl"
with open(filename, 'rb') as f:
    [azDelays, elDelays] = pkl.load(f)


fig, axs = plt.subplots(1,2,figsize=(10,5))
plt.subplots_adjust(wspace = 0.3)
plt.suptitle(f"Delay between actualPosition and encoder heads - {dayObs}")

plotRange = (0.75, 1.25)
for ii in range(4): # Encoder head
    azDels = []
    elDels = []
    for event in goodEvents:
        seqNum = event.seqNum
        key = f"{seqNum}_{ii}"
        azDels.append(azDelays[key])
        elDels.append(elDelays[key])
    axs[0].hist(azDels, bins=50, alpha=0.5, range = plotRange, edgecolor='black', label=f"Head{ii}")
    axs[1].hist(elDels, bins=50, alpha=0.5, range = plotRange, edgecolor='black', label=f"Head{ii}")
axs[0].set_title("Azimuth delays")
axs[0].set_xlim(0.8, 1.2)
axs[0].set_xlabel("Delay(seconds)")
axs[0].legend()
axs[1].set_title("Elevation delays")
axs[1].set_xlim(0.8, 1.2)
axs[1].set_xlabel("Delay(seconds)")
axs[1].legend()
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Encoder_Delay_Summary_{dayObs}.png")